# IBM Data Science Capstone Project

## Coursera Course 9

### This notebook contains the code for the execution of the capstone project


In [6]:
import pandas as pd
import numpy as np

In [7]:
print("Hello Capstone Poject Course")

Hello Capstone Poject Course
